In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
api_token = {"username":"kajolmaster","key":"dbb23ceaca2a39397c3e92d83fda1034"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:json.dump(api_token, file)

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download gowrishankarp/newspaper-text-summarization-cnn-dailymail

100% 503M/503M [00:25<00:00, 22.8MB/s]
100% 503M/503M [00:25<00:00, 20.5MB/s]


In [ ]:
!unzip newspaper-text-summarization-cnn-dailymail.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [ ]:
%%capture
!pip install transformers # Installing the transformers library (https://huggingface.co/docs/transformers/index)
!pip install datasets # Installing the datasets library (https://huggingface.co/docs/datasets/index)
!pip install evaluate # Installing the evaluate library (https://huggingface.co/docs/evaluate/main/en/index)
!pip install rouge-score # Installing rouge-score library (https://pypi.org/project/rouge-score/)
!pip install py7zr # Installing library to save zip archives (https://pypi.org/project/py7zr/)
!pip install -U accelerate

In [ ]:
# Importing Libraries

# Data Handling
import pandas as pd
import numpy as np
from datasets import Dataset, load_metric
import shutil

# Data Visualization
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
from IPython.display import display
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# Statistics & Mathematics
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import shapiro, skew, anderson, kstest, gaussian_kde,spearmanr
import math

# Hiding warnings
import warnings
warnings.filterwarnings("ignore")

# Transformers
from transformers import BartTokenizer, BartForConditionalGeneration      # BERT Tokenizer and architecture
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments         # These will help us to fine-tune our model
from transformers import pipeline                                         # Pipeline
from transformers import DataCollatorForSeq2Seq                           # DataCollator to batch the data
import torch                                                              # PyTorch
import evaluate                                                           # Hugging Face's library for model evaluation


# Other NLP libraries
from textblob import TextBlob                                             # This is going to help us fix spelling mistakes in texts
from sklearn.feature_extraction.text import TfidfVectorizer               # This is going to helps identify the most common terms in the corpus
import re                                                                 # This library allows us to clean text data
import nltk                                                               # Natural Language Toolkit
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_df=pd.read_csv("/content/cnn_dailymail/train.csv")
test_df=pd.read_csv("/content/cnn_dailymail/test.csv")
valid_df=pd.read_csv("/content/cnn_dailymail/validation.csv")

In [ ]:
train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
train_df["article"][0]

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [ ]:
train_df["highlights"][0]

'Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed .'

In [ ]:
train_df=train_df.loc[:2000]
valid_df=valid_df[:500]

In [ ]:
train_df.drop(["id"],axis=1)
valid_df.drop(["id"],axis=1)

,article,highlights
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...
...,...,...
495,Former member of 'The Real Housewives of Miami...,"Ana Quincoces, 50, was pulled over in Key Larg..."
496,Dzhokhar Tsarnaev's lawyers have been trying t...,New picture emerged on eighth day of the feder...
497,The odds of the Duke and Duchess of Cambridge ...,William Hill has slashed its odds on the new r...
498,Former England forward Gareth Hock could face ...,Four Super League clubs in draw for fifth roun...


In [ ]:
train_ds=Dataset.from_pandas(train_df)
valid_ds=Dataset.from_pandas(valid_df)

In [ ]:
train_ds

Dataset({
    features: ['id', 'article', 'highlights'],
    num_rows: 2001
})

In [ ]:
checkpoint = 'facebook/bart-large-xsum' # Model
tokenizer = BartTokenizer.from_pretrained(checkpoint) # Loading Tokenizer
model = BartForConditionalGeneration.from_pretrained(checkpoint) # Loading Model

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
valid_ds

Dataset({
    features: ['id', 'article', 'highlights'],
    num_rows: 500
})

In [ ]:
# Applying preprocess_function to the datasets
tokenized_train = train_ds.map(preprocess_function, batched=True,
                               remove_columns=['id', 'article', 'highlights']) # Removing features

tokenized_val = valid_ds.map(preprocess_function, batched=True,
                               remove_columns=['id', 'article', 'highlights']) # Removing features

Map:   0%|          | 0/2001 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2001
})

In [ ]:
# Selecting a sample from the dataset
sample = tokenized_train[0]

# Printing its features
print("input_ids:")
print(sample['input_ids'])
print("\n")
print("attention_mask:")
print(sample['attention_mask'])
print("\n")
print("sample:")
print(sample['labels'])
print("\n")

input_ids:
[0, 2765, 479, 1562, 977, 479, 29731, 7976, 14849, 1691, 35, 479, 501, 35, 1225, 12936, 6, 564, 779, 1014, 479, 1721, 479, 121, 44964, 35, 479, 379, 35, 3367, 12936, 6, 564, 779, 1014, 479, 20, 23766, 9, 5, 8044, 4019, 3643, 12074, 11, 369, 6223, 34, 4924, 2905, 2213, 9, 2352, 453, 11, 8044, 6, 2374, 286, 2258, 8, 7832, 990, 3355, 7, 5, 24426, 83, 6793, 11, 628, 772, 8, 419, 779, 4, 20, 194, 1309, 641, 34, 1167, 41, 7640, 9, 4895, 13, 1268, 54, 2922, 292, 10550, 8, 362, 42418, 4, 8163, 610, 41303, 102, 36, 7711, 43, 9, 5, 8044, 4019, 3643, 12074, 11, 369, 6223, 34, 4924, 2905, 2213, 9, 2352, 453, 11, 8044, 6, 2374, 286, 2258, 8, 7832, 990, 3355, 7, 5, 24426, 83, 479, 331, 29054, 1938, 4928, 4827, 17154, 23273, 161, 5, 810, 16, 614, 6, 53, 503, 619, 24, 18, 505, 7, 5439, 82, 7, 5, 678, 4895, 4, 20, 2269, 12074, 585, 15, 302, 14, 8163, 610, 41303, 102, 16, 602, 86, 160, 71, 145, 6443, 19, 24426, 83, 4, 20, 2269, 12074, 161, 37, 13196, 5, 7910, 149, 16194, 689, 150, 5190, 10, 1

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
metric = load_metric('rouge') # Loading ROUGE Score

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred  # Obtaining predictions and true labels

    # Decoding predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Obtaining the true labels tokens, while eliminating any possible masked token (i.e., label = -100)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]


    # Computing rouge score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()} # Extracting some results

    # Add mean-generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = 'bart_samsum',       # The output directory where the model predictions and checkpoints will be written.
    evaluation_strategy = "epoch",    #The evaluation strategy to adopt during training
    save_strategy = 'epoch',          # The checkpoint save strategy to adopt during training
    load_best_model_at_end = True,    #Whether or not to load the best model found during training at the end of training. When this option is enabled, the best checkpoint will always be saved
    metric_for_best_model = 'eval_loss', # Use in conjunction with load_best_model_at_end to specify the metric to use to compare two different models.
    seed = 42,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

In [ ]:
# Defining Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train() # Training model

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.857100,1.695360,39.292100,17.457100,27.530000,36.339700,61.674000
2,0.990200,1.789373,40.854500,18.552200,28.134500,37.944300,61.356000
3,0.779800,1.890185,40.879700,18.489500,28.287600,37.815200,61.246000


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2000, training_loss=1.2346436462402344, metrics={'train_runtime': 4219.1873, 'train_samples_per_second': 1.897, 'train_steps_per_second': 0.474, 'total_flos': 1.5272252769705984e+16, 'train_loss': 1.2346436462402344, 'epoch': 4.0})

In [ ]:
# Saving model to a custom directory
directory = "bart_finetuned_samsum"
trainer.save_model(directory)

# Saving model tokenizer
tokenizer.save_pretrained(directory)

('bart_finetuned_samsum/tokenizer_config.json',
 'bart_finetuned_samsum/special_tokens_map.json',
 'bart_finetuned_samsum/vocab.json',
 'bart_finetuned_samsum/merges.txt',
 'bart_finetuned_samsum/added_tokens.json')

In [ ]:
# Saving model in .zip format
shutil.make_archive('bart_finetuned_samsum', 'zip', '/content/bart_finetuned_samsum')
shutil.move('bart_finetuned_samsum.zip', '/content/bart_finetuned_samsum.zip')

'/content/bart_finetuned_samsum.zip'

In [ ]:
summarizer = pipeline('summarization', model = 'luisotorres/bart-finetuned-samsum')

config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
test_df.head(3)

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...


In [ ]:
test_df["article"][0]

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for sp

In [ ]:
text=test_df["article"][0]
summary = test_df["highlights"][0]
generated_summary = summarizer(text)

In [ ]:
generated_summary

[{'summary_text': 'There are growing concerns about the growing number of people travelling by plane. Some airlines offer less space for their passengers.'}]

In [ ]:
summary

'Experts question if  packed out planes are putting passengers at risk .\nU.S consumer advisory group says minimum space must be stipulated .\nSafety tests conducted on planes with more leg room than airlines offer .'